# Buffer

In [ ]:
import numpy as np
import sc3nb as scn
import ipywidgets as widgets

In [ ]:
sc = scn.startup()

## Create Buffer from np Array

In [ ]:
d0 = np.random.rand(60000,1)

In [ ]:
buf0 = sc.Buffer().load_data(d0[0:60000, 0])

In this case a default buffer with default sample rate and default insert mode is created. If you want to create a buffer with a specific sample rate or OSC insertion method etc.. Attention: the OSC insertion is only possible for small datasets (less than 1000 entries).

In [ ]:
buf01 = sc.Buffer().load_data(d0[0:600, 0], sr=5, mode="osc")

## Create Buffer from PyA
Only theorical - I don't have the pya package to test it

import pya

d1 = #create some random pya data

buf1 = sc.Buffer().load_pya(d1)

Also for PyA you can use the OSC insertion method, all the parameter mode="osc" to load_pya()

## Create Buffer of .wav File

In [ ]:
buf2 = sc.Buffer().load_file("./my_recording.wav")

The buffer method will automatically read the sample reate of the file and set it to Buffer.sr

## Allocate an empty Buffer

In [ ]:
buf3 = sc.Buffer().alloc(8*44100, sr=44100)

## Reuse existing SC buffer
If you have allready created a buffer in SC and want to have a buffer object of this existing bufnum, you can do it via:

In [ ]:
%sc b = Buffer.read(s, Platform.resourceDir +/+ "sounds/a11wlk01.wav");

In [ ]:
buf4num = %scg b.bufnum

In [ ]:
buf4 = sc.Buffer().load_existing(buf4num)

## Play Buffer

In [ ]:
buf4.play()

In [ ]:
buf4.play(rate=2)

In [ ]:
buf4.play(loop=True)

## Write buffer content to file
Write the content of a buffer into a file. By default it is a .wav File with float as sample. You can change it via parameters "header" and "sample".

In [ ]:
buf4.write("./output.wav")

## Fill buffer with values

In [ ]:
buf3.fill(0, 8*44100, 700)

## Information about the buffer
Information about the buffer object:

In [ ]:
buf3

Information about the buffer in SC
(Known bug: You have to call this method multiple times, until you've got a list with the bufnum in the first list item)

In [ ]:
buf3.query()

## Delete/ Free buffer
Set all values of the buffer to zero:

In [ ]:
buf3.zero()

Delete buffer in SC:

In [ ]:
buf3.free()